# Data Prep

**Prerequisites**

- Pandas
- Sklearn

**Outcomes**

- Be familiar with some common data prep tools: standardizing, scaling, feature encoding
- Be able to construct a sklearn pipeline that does data preparation work

## Data cleaning

- It is often said that more than 90% of a data scientist's time is spent preparing data
- That's likely an underestimate
- In order to derive useful results from a model, you need to feed the model useful data
- As the saying goes "Garbage in, garbage out"

### Start early

- The data preparation process begins before any data is actually collected!
- Being part of experiment design or the data collection process is first best
- When this is not possible, knowing as much as possible about data source will help
    - Identify potential biases    
    - Gain intuition on relationships between data
    - Know what data *isn't* there

### Types of data manipulation

- Once you have some data and have decided what to model, you will likely need to prepare that data for the model
- Some common transformations include:
    - Standariding or scaling the data
    - Feature encoding: one-hot-encoding, ordinal encoding, discritization
    - Handling missing data: imputation, filtering, 
    - Feature engineering: polynomial features, other non-linear transforms
- sklearn provides tools for all these types of pre-processing

## Pipelines

In sklearn there are two core parent classes

1. `Transformers`: transform from $X$ to $\hat{X}$
    - `.fit(X)`: performs necessary calculations to do transformation (stores results)
    - `.transform(X)`: does transform of $X$ to $\hat{X}$
1. `Estimators`: Given $X$ and $y$ data (or just $X$ for unsupervised) find model *parameters*
    - `.fit(X, y)`: compute parameters of model
    - `.predict(X)`: compute predicted $y$'s based on $X$'s that are passed in
    
> Note `.fit_transform(X)` is shorthand for first fitting, then transforming. Similarly `.fit_predict` will first fit and then generate predictions

### `sklearn.pipeline`

- Many ML tasks require multiple steps of preprocessing before passing data to model
- These are represented as transformers
- A pipeline is a 0 or more transformers and then a single Estimator
- Data is passed through transformers, in the order specified, then to estimator


### pipeline lifecycle

1. Define the pipeline `model = sklearn.pipeline.make_pipeline([trans1, trans2, ..., transN, est])`
2. Fit the model: `model.fit(X, y)`. Looks like this:
```python
X1 = trans1.fit_transform(X)
X2 = trans2.fit_transform(X1)
# ...
XN = transN.fit_transform(XNm)
est.fit(XN, y)
```
3. Generate predictions: `model.predict(x)`:
```python
x1 = trans1.transform(x)
x2 = trans2.transform(x1)
# ...
xn = transN.transform(xNm)
yhat = est.predict(xn)
```

> pipelines save you the hassle of calling `.fit` and `.transform` on all the transformers every time!

### Example

In [1]:
from sklearn import preprocessing, pipeline, linear_model
import numpy as np

In [2]:
# create some dummy data
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

y = np.array([0.1, -2.3, 1.2])

In [3]:
# define and fit transformer
trans1 = preprocessing.PolynomialFeatures(degree=3)
trans1.fit(X_train)

PolynomialFeatures(degree=3)

In [4]:
# apply transformation to training data
X1 = trans1.transform(X_train)
print(X1.shape)
X1

(3, 20)


array([[ 1.,  1., -1.,  2.,  1., -1.,  2.,  1., -2.,  4.,  1., -1.,  2.,
         1., -2.,  4., -1.,  2., -4.,  8.],
       [ 1.,  2.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1., -1.,  0.,  0., -0.,  1., -1.,  1.,  0.,  0., -0.,
         0., -0.,  0.,  1., -1.,  1., -1.]])

In [5]:
# define and fit linear model, using transformed data
linreg = linear_model.LinearRegression()
linreg.fit(X1, y)

LinearRegression()

In [6]:
# predict on training set
linreg.predict(X1)

array([ 0.1, -2.3,  1.2])

In [7]:
# let's try to evaluate on a test dataset
X_test = np.array([[2, 3, 1], [-1, -1, 0.2]])

In [8]:
# easy to go wrong...
linreg.predict(X_test)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 20 is different from 3)

In [9]:
# need to transform first...
X_test2 = trans1.transform(X_test)

In [10]:
# ... then we can predict
linreg.predict(X_test2)

array([-1.22770181,  1.06545001])

In [11]:
# easier to set up in a pipeline
model = pipeline.make_pipeline(trans1, linreg)

# single call to fit
model.fit(X_train, y)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression', LinearRegression())])

In [12]:
X_test

array([[ 2. ,  3. ,  1. ],
       [-1. , -1. ,  0.2]])

In [13]:
# single call to predict
model.predict(X_test)

array([-1.22770181,  1.06545001])

## Scaling

- Many machine learning algorithms require data to be scaled
- Sometimes, the underlying math will even assume features `X` are distributed N(0, 1)
- `sklearn.preprocessing.StandardScaler` is a routine to make each feature have mean 0 and variance 1

In [14]:
import pandas as pd

In [15]:
df = pd.read_parquet("https://css-materials.s3.amazonaws.com/ML/linear_models_2/insurance_claims_data.parquet")

In [16]:
df_numbers = df.select_dtypes([float, int])
df_strings = df.select_dtypes([object])

In [17]:
df_numbers.describe().T

,count,mean,std,min,25%,50%,75%,max
ClaimNb,678013.0,0.038946,0.205036,0.000000,0.00,0.00,0.00,4.0
Exposure,678013.0,0.528545,0.364081,0.002732,0.18,0.49,0.99,1.0
VehPower,678013.0,6.454631,2.050906,4.000000,5.00,6.00,7.00,15.0
VehAge,678013.0,7.044265,5.666232,0.000000,2.00,6.00,11.00,100.0
DrivAge,678013.0,45.499122,14.137444,18.000000,34.00,44.00,55.00,100.0
BonusMalus,678013.0,59.761502,15.636658,50.000000,50.00,50.00,64.00,230.0
Density,678013.0,1792.422405,3958.646564,1.000000,92.00,393.00,1658.00,27000.0
ClaimAmount,678013.0,76.947842,1611.088268,0.000000,0.00,0.00,0.00,200000.0
PurePremium,678013.0,341.075551,28017.959866,0.000000,0.00,0.00,0.00,18307366.0
Frequency,678013.0,0.118170,2.005521,0.000000,0.00,0.00,0.00,365.0


In [18]:
scaler = preprocessing.StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_numbers),
    index=df_numbers.index,columns=df_numbers.columns
)

In [19]:
df_scaled.describe().T

,count,mean,std,min,25%,50%,75%,max
ClaimNb,678013.0,-1.574680e-13,1.000001,-0.189948,-0.189948,-0.189948,-0.189948,19.318793
Exposure,678013.0,1.956628e-14,1.000001,-1.444220,-0.957329,-0.105868,1.267455,1.294921
VehPower,678013.0,2.444353e-13,1.000001,-1.196853,-0.709263,-0.221674,0.265916,4.166635
VehAge,678013.0,-1.176520e-13,1.000001,-1.243202,-0.890233,-0.184296,0.698125,16.405225
DrivAge,678013.0,-3.315213e-15,1.000001,-1.945128,-0.813381,-0.106039,0.672037,3.855076
BonusMalus,678013.0,-4.737003e-14,1.000001,-0.624271,-0.624271,-0.624271,0.271062,10.887149
Density,678013.0,-2.660403e-14,1.000001,-0.452534,-0.429547,-0.353511,-0.033957,6.367731
ClaimAmount,678013.0,-7.539546e-14,1.000001,-0.047761,-0.047761,-0.047761,-0.047761,124.092021
PurePremium,678013.0,3.365531e-16,1.000001,-0.012173,-0.012173,-0.012173,-0.012173,653.403692
Frequency,678013.0,-6.784668e-14,1.000001,-0.058922,-0.058922,-0.058922,-0.058922,181.938764


Notice mean and std are now (0,1) for all variables

Further Reference: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html